In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from statsmodels.formula.api import ols
from sklearn.metrics import r2_score


In [3]:
train=pd.read_excel("knn.xlsx")
test=pd.read_excel("knn.xlsx",1)

**Here I have code KNN regression from scratch, predicted the outcome for the test data and compared it with the KNN model from SKlearn library with MAPE as performance metric.**

In [8]:
#read the dataset
df=pd.read_csv("dia.csv")
df['cut'].replace({"Ideal":1,"Premium":2,"Very Good":3,"Good":4,"Fair":5},inplace=True)


In [4]:
#predict all the test values here
predicted_value=list()
pre_y=dict()
for i in range(0,test.shape[0]):
    data=list()
    data=test.iloc[i,0:3]  # take one row of the test data
    train["Distance"]=np.sqrt( ((train["X1"]-(data[0]))**2 ) +  ((train["X2"]-(data[1]))**2 )   + ((train["X3"]-(data[2]))**2 )    )
    sorted_result=train.sort_values(by="Distance")   #calculate eucilidean distance and sort the values by ascending order of distance
    predicted_value.append(sorted_result.iloc[0:3,-2].mean())
pre_y={"Pre_Y":predicted_value}
#here I am able to get the predicted Y
test["Predicted_Y"]=pd.DataFrame(pre_y)
Result=test
y_test=np.array(Result["y"])
y_pre=np.array(Result["Predicted_Y"])
mape_test = np.mean(np.abs((y_pre - y_test)/y_test)) 
print("The mape of the test data from model calculated from scratch: ",mape_test)
model=KNeighborsRegressor(n_neighbors=3)
model.fit(train[["X1","X2","X3"]],train["y"])
y_pre=model.predict(test[["X1","X2","X3"]])
print("the mape of the values from sklean: ",mean_absolute_percentage_error(y_test, y_pre))

The mape of the test data from model calculated from scratch:  0.4335604638213743
the mape of the values from sklean:  0.4335604638213743


In [9]:
#read the dataset
df=pd.read_csv("dia.csv")
df['cut'].replace({"Ideal":1,"Premium":2,"Very Good":3,"Good":4,"Fair":5},inplace=True)
#declare the features and label
X=df[["carat","depth","table","x","y","z","cut"]]
y=df["price"]

**Function to perform gradient boosting regression using decision trees and calculate its mape and also calculate the mape of gradient boosting regressor model from for the given data**

In [10]:
def gb_regressor(X,y,alpha,n,tdepth):
    for i in range(0,n+1):   
        if i==0:
            y_bar=y.mean()
            e=np.array(y-y_bar)   # initialize the errors in the first step using the mean of label values
        if i==1:
            model=DecisionTreeRegressor(max_depth=tdepth)
            model.fit(X,e)    # fit the model with the feature and error
            e_hat=model.predict(X)  # predict the values using the feature
            y_hat=y_bar+alpha*e_hat     # calculate the y_hat using the y_bar value
            e=y-y_hat   
# repeat the steps from the second iteration since the calculation of errors and y_hat would follow a loop after two iterations 
        if i>1:    
            model=DecisionTreeRegressor(max_depth=tdepth)   
            model.fit(X,e)
            e_hat=model.predict(X)
            y_hat=y_hat+alpha*e_hat
            e=y-y_hat
    #once the iterations are done we would get the final predicted values of y
    
    mdl=GradientBoostingRegressor(max_depth=tdepth,learning_rate=alpha,n_estimators=n)
    mdl.fit(X,y) #initialize the gradient boosting regressor from sklearn then fit and predict the X values
    pre_y=mdl.predict(X)
    cal_mape="The Mape of Gradient Boosting Regressor from scratch: "+str(np.mean(np.abs((y-y_hat)/y))) #here I have calculated the Mape of Gradient Boosting Algorithm which was done from scratch
    mdl_mape="The Mape of Gradient Boosting Regressor model from sklearn: "+str(np.mean(np.abs((y-pre_y)/y)))  #here I have calculated the sklearn library's gradient boosting regressor from scratch
    #prin the mape_values here
    print(cal_mape) 
    print(mdl_mape) 

In [11]:
gb_regressor(X,y,0.5,100,100)

The Mape of Gradient Boosting Regressor from scratch: 0.011643594277307814
The Mape of Gradient Boosting Regressor model from sklearn: 0.011643584340445965


From the above output the Mape values of Gradient Boosting Regressor done from scratch and the algorithm coded from scratch are matching with highly negligible difference between the MAPE values.